In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/fernandososter/RAG.git

Cloning into 'RAG'...
remote: Enumerating objects: 20549, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 20549 (delta 47), reused 22 (delta 8), pack-reused 20474 (from 2)
Receiving objects: 100% (20549/20549), 229.97 MiB | 20.04 MiB/s, done.
Resolving deltas: 100% (3390/3390), done.
Updating files: 100% (114/114), done.


In [3]:
!pip install -r /content/RAG/QLoRA-Youtube/requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 kB 40.5 MB/s eta 0:00:00


In [4]:
import os
import pandas as pd
import scipy as sp
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, set_seed
from peft import PeftModel
#from item import Item
import pickle
from peft import LoraConfig
import wandb
from huggingface_hub import login
from trl import SFTTrainer, SFTConfig
from google.colab import userdata


%matplotlib inline

In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
  if gpu_info.find('Tesla T4') >= 0:
    print("Success - Connected to a T4")
  else:
    print("NOT CONNECTED TO A T4")

Tue Nov 11 23:30:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
hf_token = userdata.get('HF_TOKEN')
wandb_api_key = userdata.get('WANDB_API_KEY')

login(hf_token, add_to_git_credential=True)

os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: fernando-soster (fernando-soster-usp-universidade-de-s-o-paulo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [7]:
PROJECT_PATH = '/content/RAG/QLoRA-Youtube/'
MODEL_CACHE_PATH = '/content/drive/MyDrive/modelos/'

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
PROJECT_NAME = "QLoRA-Youtube"
HF_USER = "fsoster"

QUANT_4_BIT = True

GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"
COLOR_MAP = {"red":RED, "orange": YELLOW, "green": GREEN}
RUN_NAME = "2025-11-11_13.04.39"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"

FINETUNED_MODEL = f"{HF_USER}/{PROJECT_RUN_NAME}"


LOG_TO_WANDB = True

os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

if LOG_TO_WANDB:
  wandb.init(project=PROJECT_NAME, name=RUN_NAME)



In [8]:
from typing import Optional
class Item:
    prompt: Optional[str] = None
    PREFIX = "Views are "
    QUESTION = "How many views for this video?"
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)

    title: str
    view_count: float

    def __init__(self, title, view_count):
        self.title = title
        self.view_count = view_count
        self.makePrompt(self.title)


    def makePrompt(self, text):
        self.prompt = f"{self.QUESTION}\n\n{text}\n\n"
        self.prompt += f"{self.PREFIX}{str(round(self.view_count))}"
        self.token_count = len(self.tokenizer.encode(self.prompt, add_special_tokens=False))



    def test_prompt(self):
        return self.prompt.split(self.PREFIX)[0] + self.PREFIX

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [9]:
df = pd.read_csv(f"{PROJECT_PATH}/dataset/youtube_video.csv")
df.head()

,video_id,title,channel_name,channel_id,view_count,like_count,comment_count,published_date,thumbnail
0,QwtOeDR-N3g,Why do Human Feet Wash up on This Beach? | Fas...,Fascinating Horror,UCFXad0mx4WxY1fXdbvtg0CQ,33955,1817,119,2025-10-03T09:15:02Z,https://i.ytimg.com/vi/QwtOeDR-N3g/hqdefault.jpg
1,K0fw1uiSGE0,"Range Rover SV (2025) - The King of the City, ...",CAR TV,UCs_gcVRMHEx5mVXfQ7icQTg,59659,1130,22,2025-10-08T09:10:46Z,https://i.ytimg.com/vi/K0fw1uiSGE0/hqdefault.jpg
2,StuKQpzfyjI,Bruce Wayne Becomes Batman - NEW Gotham City U...,Just4fun290,UCdPG86Ahrf9A8YzBnFDAaEg,8658,89,0,2025-10-04T15:00:06Z,https://i.ytimg.com/vi/StuKQpzfyjI/hqdefault.jpg
3,C91854vbs8I,How the music of Mario Kart has evolved,Thomas Game Docs,UCyy7dZhgfeMMctSoo3wDXlQ,161045,5391,239,2025-05-08T15:32:15Z,https://i.ytimg.com/vi/C91854vbs8I/hqdefault.jpg
4,1DaHVSmx160,My bulldog loves mess... you can see that )))),Enzo English Bulldog,UCkk_2IfQ52OJcCUXOrJBtnw,7620,94,24,2021-08-12T15:33:31Z,https://i.ytimg.com/vi/1DaHVSmx160/hqdefault.jpg


In [10]:
columns = ["title", "view_count"]
items = [Item(**item) for item in df[columns].to_dict(orient="records")]
test = items[:round(len(items)*0.3)]
train = items[round(len(items)*0.3):]

In [11]:
'''
pkl_train_file = 'dataset/train.pkl'
pkl_test_file = 'dataset/test.pkl'

if os.path.exists(pkl_train_file):
  with open(pkl_train_file, "rb") as f:
        train = pickle.load(f)

  with open(pkl_test_file, "rb") as f:
        test = pickle.load(f)

else:
    columns = ["title", "view_count"]
    items = [Item(**item) for item in df[columns].to_dict(orient="records")]
    test = items[:round(len(items)*0.3)]
    train = items[round(len(items)*0.3):]
    with open(pkl_train_file, 'wb') as file:
        pickle.dump(train, file)

    with open(pkl_test_file, 'wb') as file:
        pickle.dump(test, file)
'''

'\npkl_train_file = \'dataset/train.pkl\'\npkl_test_file = \'dataset/test.pkl\'\n\nif os.path.exists(pkl_train_file):\n  with open(pkl_train_file, "rb") as f:\n        train = pickle.load(f)\n\n  with open(pkl_test_file, "rb") as f:\n        test = pickle.load(f)\n\nelse:\n    columns = ["title", "view_count"]\n    items = [Item(**item) for item in df[columns].to_dict(orient="records")]\n    test = items[:round(len(items)*0.3)]\n    train = items[round(len(items)*0.3):]\n    with open(pkl_train_file, \'wb\') as file:\n        pickle.dump(train, file)\n\n    with open(pkl_test_file, \'wb\') as file:\n        pickle.dump(test, file)\n'

In [12]:
len(train), len(test)

(108968, 46701)

In [13]:

if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

In [14]:
set_seed(42)
prompt = train[0].prompt
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
    cache_dir=MODEL_CACHE_PATH
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:.1f} GB")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Memory footprint: 5.6 GB


In [15]:
LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]
LORA_DROPOUT = 0.1

lora_parameters = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM", # Specifies we're doing causal language modeling
)

In [16]:
# 📦 Training Setup:
EPOCHS = 1
BATCH_SIZE = 16                     # A100 GPU can go up to 16
GRADIENT_ACCUMULATION_STEPS = 2
MAX_SEQUENCE_LENGTH = 182          # Max token length per input

# ⚙️ Optimization:
LEARNING_RATE = 1e-4
LR_SCHEDULER_TYPE = 'cosine'
WARMUP_RATIO = 0.03
OPTIMIZER = "paged_adamw_32bit"

# 💾 Checkpointing & Logging:
SAVE_STEPS = 200        # Checkpoint
STEPS = 20              # Log every 20 steps
save_total_limit = 10   # Keep latest 10 only


LOG_TO_WANDB = True

HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"

train_parameters = SFTConfig(
    # Output & Run
    output_dir=PROJECT_RUN_NAME,
    run_name=RUN_NAME,
    dataset_text_field="text",
    #max_seq_length=MAX_SEQUENCE_LENGTH,

    # Training
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    max_steps=-1,
    group_by_length=True,

    # Evaluation
    eval_strategy="steps",
    eval_steps=STEPS,
    per_device_eval_batch_size=1,

    # Optimization
    learning_rate=LEARNING_RATE,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    warmup_ratio=WARMUP_RATIO,
    optim=OPTIMIZER,
    weight_decay=0.001,
    max_grad_norm=0.3,

    # Precision
    fp16=False,
    bf16=True,

    # Logging & Saving
    logging_steps=STEPS,            # See loss after each {STEP} batches
    save_strategy="steps",
    save_steps=SAVE_STEPS,          # Model Checkpointed locally
    save_total_limit=save_total_limit,
    report_to="wandb" if LOG_TO_WANDB else None,

    # Hub
    push_to_hub=True,
    hub_strategy="end",  # Only push once, at the end
    load_best_model_at_end=True, # Loads the best eval_loss checkpoint
    metric_for_best_model="eval_loss", # Monitors eval_loss
    greater_is_better=False, # Lower eval_loss = better model
)


In [19]:
def item_to_dict(item):
    return {
        "prompt": item.prompt,
        "completion": str(item.view_count),
    }

train_data = [item_to_dict(it) for it in train]
test_data = [item_to_dict(it) for it in test]


In [ ]:
print(f"{len(train_data)},{len(test_data)}" )
train_data = train_data[len(train_data)*0.1:]
test_data = test_data[len(test_data)*0.1:]
print(f"{len(train_data)},{len(test_data)}" )

In [ ]:

from datasets import Dataset

train_dataset = Dataset.from_list(train_data)
tes_dataset = Dataset.from_list(test_data)

In [18]:
# The latest version of trl is showing a warning about labels - please ignore this warning
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=lora_parameters,    # QLoRA config
    args=train_parameters,          # SFTConfig
    #data_collator=collator,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=5)] # Early stop if no val improvement for 5 steps
)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

TypeError: argument of type 'Item' is not iterable

In [ ]:
fine_tuning.train()
print(f"✅ Best model pushed to HF Hub: {HUB_MODEL_NAME}")

In [ ]:
for item in train:
    inputs = tokenizer.encode(item.prompt, return_tensors="pt").to("cuda")
    attention_mask = torch.ones(inputs.shape, device="cuda")
    outputs = base_model.generate(inputs, max_new_tokens=4, attention_mask=attention_mask, num_return_sequences=1)
    response = tokenizer.decode(outputs[0])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="cpu",
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id


fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)

print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e6:.1f} MB")

In [ ]:
fine_tuned_model